In [5]:
from nba_api.stats.static import players, teams as teams_static
from nba_api.stats.endpoints import playergamelog, commonallplayers, commonteamroster
from tqdm.auto import tqdm
import pandas as pd
import time
import random


In [ ]:
# получаем всех игроков для сезона
def get_all_players(season):
    teams_list = teams_static.get_teams()
    player_map = {}
    for team in tqdm(teams_list, desc ='Processing Teams'):
        team_id = team.get('id')
        try:
            time.sleep(1.6)
            roster = commonteamroster.CommonTeamRoster(team_id=team_id, season=season)
            roster_df = roster.get_data_frames()[0]
        except Exception as e:
            print(f"Error fetching roster for team ID {team['full_name']}: {e}")
            continue
        time.sleep(1.6)
        for _, row in roster_df.iterrows():
            player_map[row['PLAYER_ID']] = {
            'PLAYER_NAME' : row['PLAYER'],
            'TEAM_ID' : team_id,
            'TEAM_NAME' : team.get('full_name'),
            'POSITION' : row['POSITION'],
            'HEIGHT' : row['HEIGHT'],
            'WEIGHT' : row['WEIGHT'],
            'AGE' : row['AGE']
            }
    df_players = pd.DataFrame.from_dict(player_map, orient='index').reset_index().rename(columns={'index':'PLAYER_ID'})
    return df_players

# получаем лог игроков для сезона
def get_player_game_logs(player_ids, season):
    all_game_logs = []
    for player_id in tqdm(player_ids, desc='Processing Players'):
        try:
            time.sleep(1.6)
            gamelog = playergamelog.PlayerGameLog(player_id=player_id, season=season)
            gamelog_df = gamelog.get_data_frames()[0]
            all_game_logs.append(gamelog_df)
        except Exception as e:
            print(f"Error fetching game log for player ID {player_id}: {e}")
            continue
    if all_game_logs:
        combined_game_logs = pd.concat(all_game_logs, ignore_index=True)
    else:
        combined_game_logs = pd.DataFrame()
    return combined_game_logs


In [9]:
players_2025 = get_all_players('2024-25')
players_2025

Processing Teams:   0%|          | 0/30 [00:00<?, ?it/s]

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,POSITION,HEIGHT,WEIGHT,AGE
0,1631210,Jacob Toppin,1610612737,Atlanta Hawks,F,6-8,200,25.0
1,1630552,Jalen Johnson,1610612737,Atlanta Hawks,F,6-8,219,23.0
2,1630811,Keaton Wallace,1610612737,Atlanta Hawks,G,6-3,185,26.0
3,1627747,Caris LeVert,1610612737,Atlanta Hawks,G,6-6,205,30.0
4,1641723,Kobe Bufkin,1610612737,Atlanta Hawks,G,6-4,195,21.0
...,...,...,...,...,...,...,...,...
529,1641706,Brandon Miller,1610612766,Charlotte Hornets,F,6-7,200,22.0
530,1642354,KJ Simpson,1610612766,Charlotte Hornets,G,6-0,189,22.0
531,203552,Seth Curry,1610612766,Charlotte Hornets,G,6-1,185,34.0
532,1642275,Tidjane Salaün,1610612766,Charlotte Hornets,F,6-8,207,19.0


In [19]:
all_game_logs = []
for player_id in tqdm(players_2025['PLAYER_ID'], desc='Processing Players'):
    try:
        time.sleep(1.6)
        gamelog = playergamelog.PlayerGameLog(player_id=player_id, season='2024-25')
        gamelog_df = gamelog.get_data_frames()[0]
        all_game_logs.append(gamelog_df)
    except Exception as e:
        print(f"Error fetching game log for player ID {player_id}: {e}")
        continue
combined_game_logs_2025 = pd.concat(all_game_logs, ignore_index=True)
combined_game_logs_2025


Processing Players:   0%|          | 0/534 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
pd.options.display.max_columns = None

combined_game_logs_2025['GAME_DATE']  = pd.to_datetime(combined_game_logs_2025['GAME_DATE'])
combined_game_logs_2025.set_index('GAME_DATE', inplace=True)

In [38]:
# features engineering

combined_game_logs_2025.sort_values(by=['Player_ID', 'GAME_DATE'], inplace=True)

stats = ['PTS', 'AST', 'REB', 'STL', 'BLK', 'TOV', 'FG_PCT', 'FG3_PCT', 'FT_PCT', 'PLUS_MINUS']
for stat in stats:
    combined_game_logs_2025[f'PREV_10_{stat}'] = combined_game_logs_2025.groupby('Player_ID')[stat].transform(lambda x: x.shift(1).rolling(window=10, min_periods=10).mean())

for stat in stats:
    combined_game_logs_2025[f'PREV_3_{stat}'] = combined_game_logs_2025.groupby('Player_ID')[stat].transform(lambda x: x.shift(1).rolling(window=10, min_periods=10).mean())

for stat in stats:
    combined_game_logs_2025[f'last_v_{stat}'] = combined_game_logs_2025.groupby('Player_ID')[stat].shift(1)

combined_game_logs_2025['stdt PTS 10'] = combined_game_logs_2025.groupby('Player_ID')['PTS'].transform(lambda x: x.shift(1).rolling(window=10, min_periods=10).std())  
combined_game_logs_2025['stdt PTS 3'] = combined_game_logs_2025.groupby('Player_ID')['PTS'].transform(lambda x: x.shift(1).rolling(window=3, min_periods=3).std())      

combined_game_logs_2025 = combined_game_logs_2025.reset_index(drop=True)


In [39]:
combined_game_logs_2025.head(20)

,index,GAME_DATE,SEASON_ID,Player_ID,Game_ID,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE,PREV_PTS,PREV_AST,PREV_REB,PREV_STL,PREV_BLK,PREV_TOV,PREV_FG_PCT,PREV_FG3_PCT,PREV_FT_PCT,PREV_PLUS_MINUS,PREV_10_PTS,PREV_10_AST,PREV_10_REB,PREV_10_STL,PREV_10_BLK,PREV_10_TOV,PREV_10_FG_PCT,PREV_10_FG3_PCT,PREV_10_FT_PCT,PREV_10_PLUS_MINUS,PREV_3_PTS,PREV_3_AST,PREV_3_REB,PREV_3_STL,PREV_3_BLK,PREV_3_TOV,PREV_3_FG_PCT,PREV_3_FG3_PCT,PREV_3_FT_PCT,PREV_3_PLUS_MINUS,last_v_PTS,last_v_AST,last_v_REB,last_v_STL,last_v_BLK,last_v_TOV,last_v_FG_PCT,last_v_FG3_PCT,last_v_FT_PCT,last_v_PLUS_MINUS,stdt PTS 10,stdt PTS 3
0,0,2024-10-22,22024,2544,0022400062,LAL vs. MIN,W,35,7,16,0.438,1,4,0.250,1,1,1.000,0,5,5,4,0,2,2,3,16,-6,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2024-10-25,22024,2544,0022400085,LAL vs. PHX,W,35,7,13,0.538,2,5,0.400,5,8,0.625,0,4,4,8,0,0,2,1,21,14,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,4,5,0,2,2,0.438,0.250,1.000,-6,NaN,NaN
2,2,2024-10-26,22024,2544,0022400096,LAL vs. SAC,W,34,12,22,0.545,4,9,0.444,4,4,1.000,0,14,14,10,0,1,5,3,32,13,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,8,4,0,0,2,0.538,0.400,0.625,14,NaN,NaN
3,3,2024-10-28,22024,2544,0022400111,LAL @ PHX,L,36,3,14,0.214,2,5,0.400,3,6,0.500,0,5,5,8,1,0,2,1,11,-17,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32,10,14,0,1,5,0.545,0.444,1.000,13,NaN,8.185353
4,4,2024-10-30,22024,2544,0022400118,LAL @ CLE,L,29,9,13,0.692,1,3,0.333,7,7,1.000,1,5,6,3,0,0,6,2,26,-17,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,8,5,1,0,2,0.214,0.400,0.500,-17,NaN,10.503968
5,5,2024-11-01,22024,2544,0022400137,LAL @ TOR,W,36,9,17,0.529,5,8,0.625,4,6,0.667,0,6,6,10,1,0,0,1,27,11,1,21.2,6.6,6.8,0.2,0.6,3.4,0.4854,0.3654,0.8250,-2.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,3,6,0,0,6,0.692,0.333,1.000,-17,NaN,10.816654
6,6,2024-11-04,22024,2544,0022400156,LAL @ DET,L,40,7,16,0.438,1,2,0.500,5,5,1.000,1,7,8,11,2,0,1,1,20,-16,1,23.4,7.8,7.0,0.4,0.2,3.0,0.5036,0.4404,0.7584,0.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27,10,6,1,0,0,0.529,0.625,0.667,11,NaN,8.962886
7,7,2024-11-06,22024,2544,0022400174,LAL @ MEM,L,35,15,24,0.625,6,11,0.545,3,4,0.750,0,7,7,6,1,0,5,2,39,-21,1,23.2,8.4,7.8,0.8,0.2,2.8,0.4836,0.4604,0.8334,-5.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20,11,8,2,0,1,0.438,0.500,1.000,-16,NaN,3.785939
8,8,2024-11-08,22024,2544,0022400195,LAL vs. PHI,W,34,9,14,0.643,2,4,0.500,1,1,1.000,0,12,12,13,0,3,2,0,21,12,1,24.6,7.6,6.4,1.0,0.0,2.8,0.4996,0.4806,0.7834,-12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39,6,7,1,0,5,0.625,0.545,0.750,-21,NaN,9.609024
9,9,2024-11-10,22024,2544,0022400211,LAL vs. TOR,W,35,6,14,0.429,0,3,0.000,7,7,1.000,2,8,10,16,1,0,3,0,19,13,1,26.6,8.6,7.8,0.8,0.6,2.8,0.5854,0.5006,0.8834,-6.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,13,12,0,3,2,0.643,0.500,1.000,12,NaN,10.692677


,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,POSITION,HEIGHT,WEIGHT,AGE
0,1631210,Jacob Toppin,1610612737,Atlanta Hawks,F,6-8,200,25.0
1,1630552,Jalen Johnson,1610612737,Atlanta Hawks,F,6-8,219,23.0
2,1630811,Keaton Wallace,1610612737,Atlanta Hawks,G,6-3,185,26.0
3,1627747,Caris LeVert,1610612737,Atlanta Hawks,G,6-6,205,30.0
4,1641723,Kobe Bufkin,1610612737,Atlanta Hawks,G,6-4,195,21.0
...,...,...,...,...,...,...,...,...
529,1641706,Brandon Miller,1610612766,Charlotte Hornets,F,6-7,200,22.0
530,1642354,KJ Simpson,1610612766,Charlotte Hornets,G,6-0,189,22.0
531,203552,Seth Curry,1610612766,Charlotte Hornets,G,6-1,185,34.0
532,1642275,Tidjane Salaün,1610612766,Charlotte Hornets,F,6-8,207,19.0
